In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(in_channels= 3, out_channels=96, kernel_size = (11,11), stride= 4)
        self.norm1 = nn.LocalResponseNorm(5, k=2, alpha=0.00004, beta=0.75) #k = 2, n = 5, α = 10−4, and β = 0.75.
        self.pool = nn.MaxPool2d(2,2)

        self.conv2 = nn.Conv2d(96, 256, 5)
        self.conv3 = nn.Conv2d(256, 384, 3)
        self.conv4 = nn.Conv2d(384, 384, 3)
        self.conv5 = nn.Conv2d(384, 256, 3)
        self.fc1 = nn.Linear((256 * 6 * 6), 4096) #256 * 6 * 6) to flatten
        self.fc2 = nn.Linear(4096,4096)
        self.fc3 = nn.Linear(4096,1000)

